# Train ConvGNP

- train it as a off-grid model

In [1]:
import torch

# torch needs to be specified before otherwise issues with model
import deepsensor.torch
import deepsensor

# from deepsensor.model import ConvNP
from deepsensor.train import Trainer, set_gpu_default_device

# load my own function
from kb_utils import construct_circ_and_linear_time_ds

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from tqdm.notebook import tqdm # updated command to avoid warnings

import logging
logging.captureWarnings(True)

/home/kim/grace_convNP/.conda/lib/python3.11/site-packages/fdm/fdm.py:38: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  coefs = mat.inv()[:, deriv] * np.math.factorial(deriv)
/home/kim/grace_convNP/.conda/lib/python3.11/site-packages/fdm/fdm.py:44: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  / np.math.factorial(order)
/home/kim/grace_convNP/.conda/lib/python3.11/site-packages/fdm/fdm.py:38: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (Deprecated Numpy 1.25). Replace usages of `np.math` with `math`
  coefs = mat.inv()[:, deriv] * np.math.factorial(deriv)
/home/kim/grace_convNP/.conda/lib/python3.11/site-packages/fdm/fdm.py:44: DeprecationWarning: `np.math` is a deprecated alias for the standard library `math` module (D

In [2]:
# Define device
print(torch.get_default_device())
deepsensor.train.set_gpu_default_device()
print()
print("Device now:")
print("Device", torch.get_default_device())

cpu

Device now:
cuda:0


In [3]:
# load xarray DataSet
gmb_xds = xr.open_dataset(
    "/home/kim/grace_convNP/data/GRACE_COST-G/GRAVIS-3_2002095-2023334_COSTG_0100_AIS_GRID_TUD_0003.nc")

In [4]:
gmb_xds.dropna()

<xarray.Dataset>
Dimensions:   (x: 117, y: 97, time: 227)
Coordinates:
  * x         (x) float64 -2.9e+06 -2.85e+06 -2.8e+06 ... 2.85e+06 2.9e+06
  * y         (y) float64 -2.4e+06 -2.35e+06 -2.3e+06 ... 2.35e+06 2.4e+06
  * time      (time) datetime64[ns] 2002-04-18 2002-05-10 ... 2023-11-16
Data variables:
    time_dec  (time) float64 ...
    lon       (y, x) float64 ...
    lat       (y, x) float64 ...
    dm        (time, y, x) float64 ...
    area      (y, x) float64 ...
    crs       |S1 ...
Attributes: (12/16)
    title:                Gridded AIS Mass Changes
    institution:          TU Dresden, Chair of Geodetic Earth System Research
    authors:              Andreas Groh, Martin Horwath, Thorben Döhne
    project:              GravIS GRACE/GRACE-FO Level-3 Products
    source:               Monthly COST-G RL01 GRACE/GRACE-FO Level-2B product...
    summary:              GRACE/GRACE-FO derived time series of gridded Antar...
    ...                   ...
    product_version:      0003
    doi:                  10.5880/COST-G.GRAVIS_01_L3_ICE
    netCDF_version:       netCDF-4_classic
    conventions:          CF-1.6
    date_created:         2024-04-05
    contact:              thorben.doehne@tu-dresden.de

In [ ]:
# Create copy of xarray (data array) dm (delta mass) for mask
# any time slice will do
mask_xda = gmb_xds.dm["time" == "2002-04-18T00:00:00.000000000"].copy()

# rename variable name mask_xda.name
mask_xda = mask_xda.rename("land_mask")

# Remove time as (index: already removed) and as coordinate (unused coordinate)
# mask_xda = mask_xda.drop_indexes('time') # drops it from index
mask_xda = mask_xda.drop_vars("time")  # drops it in coordinates
# mask_xda = mask_xda.reset_index(("y", "x"))

# Set to 1 (land) if there are no nan
mask_xda.values[np.isnan(mask_xda.values) == False] = 1
mask_xda.values[np.isnan(mask_xda.values) == True] = 0

mask_xda

In [8]:
dates = gmb_xds.dm.time.values

# my own function
doy_xds = construct_circ_and_linear_time_ds(dates, freq="D")

In [ ]:
# (1) Data Processor
data_processor = deepsensor.data.DataProcessor(
    time_name = "time", 
    x1_name = "y", 
    x2_name = "x")

# naming indicates the proprocessing step applied
gmb_pp = data_processor(gmb_xds.dm, method = "mean_std")

mask_pp = data_processor(mask_xda.to_dataset(), method = "min_max") # [-1, 1]

# Assign after preprocessing
mask_pp["doy_cos_D"] = doy_xds["cos_D"]
mask_pp["doy_sin_D"] = doy_xds["sin_D"]
mask_pp["doy_year"] = doy_xds["year"]

# (2) task loader
task_loader = deepsensor.data.TaskLoader(
    context = [gmb_pp, mask_pp],
    target = [gmb_pp],
)

# (3) model folder with params
deepsensor_folder = "models/deepsensor_config"
# model_folder = "../model/"